In [1]:
#from gensim import utils
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
train_data = pd.read_csv('data/IMDB_review_train.csv')
test_data = pd.read_csv('data/IMDB_review_test.csv')
train_texts = list(train_data.text.values)
train_labels = list(train_data.sentiment.values)
test_texts = list(test_data.text.values)
test_labels = list(test_data.sentiment.values)

## Text to Numbers

In [3]:
#from tensorflow.contrib import learn

In [4]:
#MAX_DOCUMENT_LENGTH = 800
##vocab_processor = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH, min_frequency=2) # tensorflow提供的工具，将数据填充为最大长度，默认0填充
##train_sent_idx = np.array(list(vocab_processor.fit_transf
#                               orm(train_texts)))
#test_sent_idx = np.array(list(vocab_processor.transform(test_texts)))

In [5]:
#n_words = len(vocab_processor.vocabulary_)
#n_words

In [6]:
from tools.text_preprocess import text_clean
tc = text_clean(train_texts, is_multiprocess=False)
train_processed = tc.proceed()
tc = text_clean(test_texts, is_multiprocess=False)
test_processed = tc.proceed()

Start to process....
Processing Finished! Timing:  877.151
Start to process....
Processing Finished! Timing:  875.277


## Save the data

In [18]:
import pickle
with open('data/intermediate_data/training.pk', 'wb') as f:
    pickle.dump(train_processed, f, -1)
with open('data/intermediate_data/testing.pk', 'wb') as f:
    pickle.dump(test_processed, f, -1)

In [19]:
data = {'text': train_processed, 'sentiment':train_labels}
temp =  pd.DataFrame(data)
temp.to_csv('data/intermediate_data/train_processed.csv', index=False)
data = {'text': test_processed, 'sentiment':test_labels}
temp =  pd.DataFrame(data)
temp.to_csv('data/intermediate_data/test_processed.csv', index=False)

In [5]:
train_data = pd.read_csv('data/intermediate_data/train_processed.csv')
test_data = pd.read_csv('data/intermediate_data/train_processed.csv')
train_processed = list(train_data.text.values)
train_labels = list(train_data.sentiment.values)
test_processed = list(test_data.text.values)
test_labels = list(test_data.sentiment.values)

In [6]:
words = []
for item in train_processed:
    words.extend(item.split())

In [7]:
from collections import Counter
word_freq = Counter(words)

In [8]:
len(word_freq)

60008

In [9]:
from tools.text_hier_split import sent2words
from tools.token_idx_map import token2idx
sw_train = sent2words(train_processed)
sw_test = sent2words(test_processed)
train_sent_words = sw_train.proceed()
test_sent_words = sw_test.proceed()
ti = token2idx(train_sent_words, 30000)
train_sent_idx = ti.proceed()
test_sent_idx = ti.map_text_idx(test_sent_words, ignore_sent=True)

Start mapping words to IDs....
Processing Finished! Timing:  3.332


In [10]:
n_words = len(ti.get_vocab())
print('Total words: %d' % n_words)

Total words: 30001


In [11]:
from tools.sample_generator import generate_samples
MAX_DOCUMENT_LENGTH = 800
gs_train = generate_samples(train_sent_idx, train_labels, MAX_DOCUMENT_LENGTH)
gs_test = generate_samples(test_sent_idx, test_labels, MAX_DOCUMENT_LENGTH)

## Settings

In [12]:
class trainConfig:
    vocab_size = n_words
    max_doc_len = MAX_DOCUMENT_LENGTH
    label_size = 2
    embed_size = 64
    hidden_size = 250
    batch_size = 64
    layer_size = 2
    
class testConfig:
    vocab_size = n_words
    max_doc_len = MAX_DOCUMENT_LENGTH
    label_size = 2
    embed_size = 64
    hidden_size = 250
    batch_size = 64
    layer_size = 2
    
class singleConfig:
    vocab_size = n_words
    max_doc_len = MAX_DOCUMENT_LENGTH
    label_size = 2
    embed_size = 64
    hidden_size = 250#hidden size for hidden state of rnn
    batch_size = 1
    layer_size = 2

In [13]:
train_chunk_num = int(len(train_processed)/trainConfig.batch_size)
test_chunk_num = int(len(test_processed)/trainConfig.batch_size)
remain_num = len(test_processed) - trainConfig.batch_size*test_chunk_num
remain_num

40

## MultiChannelCNN

In [14]:
import functools
import tensorflow as tf
from tensorflow.contrib.layers.python.layers import encoders
def lazy_property(function):
    attribute = '_cache_' + function.__name__

    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            setattr(self, attribute, function(self))
        return getattr(self, attribute)

    return decorator

In [15]:
from tensorflow.contrib import rnn
class CNN_Model:
    def __init__(self, config, x, y, lengths, is_training=True):
        self.x = x
        self.y = y
        self.vocab_size = config.vocab_size
        self.embed_size = config.embed_size
        self.hidden_size = config.hidden_size
        self.label_size = config.label_size
        self.batch_size = config.batch_size
        self.lengths = lengths
        self.max_doc_len = config.max_doc_len
        self.is_training = is_training
        self.predict
        if is_training:
            self.optimize
        print('Model Initialized!')
    
    @lazy_property
    def cost(self):
        logits = self.inference
        targets = tf.one_hot(self.y, self.label_size, 1, 0)
        targets = tf.cast(targets, tf.float32)
        #Note  tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=activation)
        loss = tf.losses.softmax_cross_entropy(targets, logits)
        return loss
    
    @lazy_property
    def predict(self):
        logits = self.inference
        #probs = tf.nn.softmax(logits)
        predictions = tf.argmax(logits, 1)
        return predictions
    
    @lazy_property
    def correct_num(self):
        prediction = self.predict
        targets = tf.reshape(self.y, [-1])
        targets = tf.cast(targets, tf.int64)
        correct_prediction = tf.equal(prediction, targets)
        correct_num = tf.reduce_sum(tf.cast(correct_prediction, "float"))
        return correct_num
    
    @lazy_property
    def optimize(self):
        with tf.variable_scope('optimizer'):
            cost = self.cost
        #with tf.name_scope('Optimizer'):
            #self._learning_rate = tf.Variable(0.0, trainable=False)
            train_op = tf.train.AdamOptimizer(0.0001).minimize(cost)
            #train_op = tf.train.AdamOptimizer(self._learning_rate).minimize(cost)
            #tvars = tf.trainable_variables()
            #grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), 6)
            #optimizer = tf.train.AdamOptimizer(self._learning_rate)
            #train_op = optimizer.apply_gradients(zip(grads, tvars))
        return train_op
    
    @lazy_property
    def inference(self):
        #Create embedding matrix
        with tf.device("/cpu:0"):
            embeddings = tf.get_variable('embedding', [self.vocab_size,  self.embed_size])
            inputs = tf.nn.embedding_lookup(embeddings, self.x)
        if self.is_training:
            #Batch_size, word_length, embed_size
            inputs = tf.nn.dropout(inputs, 0.5)
        #Mask those padding words as 0, others as 11
        mask = tf.sequence_mask(self.lengths, MAX_DOCUMENT_LENGTH)
        mask = tf.cast(mask, dtype=tf.float32)
        #Duplicate binary values for the embeddings
        #Those embeddings for the padding words are masked
        #Broadcast embed_size times
        mask_transient = tf.tile(mask, [1, self.embed_size])
        mask_embed = tf.transpose(tf.reshape(mask_transient, [self.batch_size, 
                                                              self.embed_size,
                                                MAX_DOCUMENT_LENGTH]), [0, 2, 1])
        mask_inputs = tf.multiply(mask_embed, inputs)
        #Expand the dim to cater to CNN
        #Batch_size, word_length, embed_size, 1
        intputs_expanded = tf.expand_dims(mask_inputs, -1)
        #print(intputs_expanded)
        
        #Three kinds of convolutional kernels, with kernel size 2, 3, 4
        filter_sizes = [2, 3, 4]
        num_filters = 128
        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.variable_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                h = tf.layers.conv2d(intputs_expanded, num_filters,
                                       kernel_size=(filter_size, self.embed_size),
                                       strides=(1, 1), padding='valid',
                                        activation=tf.nn.relu)
    
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, filter_size, 
                           1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                #Swap the last two dimensions
                pooled = tf.transpose(pooled, [0, 1, 3, 2])
                #Second conv layer
                h = tf.layers.conv2d(pooled, num_filters,
                                       kernel_size=(filter_size, num_filters),
                                       strides=(1, 1), padding='valid',
                                        activation=tf.nn.relu)
    
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, MAX_DOCUMENT_LENGTH - 3*filter_size + 3, 
                           1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                
        
        
                pooled_outputs.append(pooled)
            #self.weights.append(W)
            
        
        num_filters_total = num_filters * len(filter_sizes)
        h_pool = tf.concat(pooled_outputs, 3)
        h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])
        
        if self.is_training:
            h_pool_flat = tf.nn.dropout(h_pool_flat, 0.5)
        
        
        with tf.variable_scope('output'):
            logits = tf.layers.dense(h_pool_flat, self.label_size, 
                                     kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003))
            #weights = tf.get_variable('weights', [num_filters_total, self.label_size], dtype=tf.float32)
            #biases = tf.get_variable('biases', [self.label_size], dtype=tf.float32)
        
        return logits
    
    @property
    def learningRate(self):
        return self._learning_rate
        

In [16]:
graph_cnn = tf.Graph()
#Create models for training and testing data
with graph_cnn.as_default():
    initializer = tf.random_uniform_initializer(-0.02, 0.02)
    with tf.name_scope('train'):
        train_data = tf.placeholder(tf.int32, [trainConfig.batch_size, MAX_DOCUMENT_LENGTH])
        train_label = tf.placeholder(tf.int32, [trainConfig.batch_size])
        train_lengths = tf.placeholder(tf.int32, [trainConfig.batch_size])
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            train_model = CNN_Model(trainConfig, train_data, train_label, train_lengths)
            saver=tf.train.Saver()
    with tf.name_scope('test'):
        test_data = tf.placeholder(tf.int32, [testConfig.batch_size, None])
        test_label = tf.placeholder(tf.int32, [testConfig.batch_size])
        test_lengths = tf.placeholder(tf.int32, [testConfig.batch_size])
        single_data = tf.placeholder(tf.int32, [singleConfig.batch_size, None])
        single_label = tf.placeholder(tf.int32, [singleConfig.batch_size])
        single_lengths = tf.placeholder(tf.int32, [singleConfig.batch_size])
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            test_model = CNN_Model(testConfig, test_data, test_label, test_lengths, False)
            single_model = CNN_Model(singleConfig, single_data, single_label, single_lengths, False)

Model Initialized!
Model Initialized!
Model Initialized!


In [17]:
gs_train = generate_samples(train_sent_idx, train_labels, MAX_DOCUMENT_LENGTH)
gs_test = generate_samples(test_sent_idx, test_labels, MAX_DOCUMENT_LENGTH)

In [19]:
import time, os
epochs = 5
#train_chunk_num = 10
file = "ckpt_cnn/cnn.ckpt"
with tf.Session(graph=graph_cnn) as sess:
    #Initialize parameters
    init = tf.global_variables_initializer()
   
    if os.path.exists("ckpt_cnn/cnn.ckpt.index"):
        saver.restore(sess, file)
    else:
         sess.run(init)
    start_time = time.time()
    for m in range(epochs):
        for i in range(train_chunk_num):
            #sess.run(tf.assign(learning_rate, 0.002*((0.98)**m)))
            x, y, lengths, _ = gs_train.generate_batch(trainConfig.batch_size)
            feed_dict = {train_data:x, train_label:y, train_lengths:lengths}
            l, _ = sess.run([train_model.cost, train_model.optimize], feed_dict=feed_dict)
            if i%100 == 0:
                print('Loss:', round(l, 4))
        end_time = time.time()
        print('Epoch', m, 'time:{:.2f}'.format(end_time - start_time))
        start_time = end_time
    saver.save(sess,'ckpt_cnn/cnn.ckpt')
    #Calculate Testing Accuracy
    print('Testing...')
    count = 0
    gs_test = generate_samples(test_sent_idx, test_labels, MAX_DOCUMENT_LENGTH)
    for _ in range(test_chunk_num):
        #Traverse each data
        x, y, lengths, _ = gs_test.generate_batch(testConfig.batch_size, False)
        feed_dict = {test_data:x, test_label:y, test_lengths:lengths}
        n = sess.run(test_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    for _ in range(remain_num):
        #Traverse each data
        x, y, lengths, _ = gs_test.generate_batch(1, False)
        feed_dict = {single_data:x, single_label:y, single_lengths:lengths}
        n = sess.run(single_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    end_time = time.time()
    print('Testing Time:{:.2f}'.format(end_time - start_time))
    print(count*1.0/len(test_texts))  

Loss: 0.6931
Loss: 0.6939
Loss: 0.6691
Loss: 0.5977
Epoch 0 time:21.78
Loss: 0.5484
Loss: 0.4285
Loss: 0.4162
Loss: 0.3698
Epoch 1 time:21.01
Loss: 0.4879
Loss: 0.5288
Loss: 0.5037
Loss: 0.4314
Epoch 2 time:21.09
Loss: 0.5402
Loss: 0.3839
Loss: 0.3637
Loss: 0.343
Epoch 3 time:21.10
Loss: 0.307
Loss: 0.3807
Loss: 0.3091
Loss: 0.2428
Epoch 4 time:21.25
Testing...
Test Samples come to an end!
Testing Time:6.35
0.8894
